# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import gmaps.datasets
import os
import json

# Import API key
from api_keys import g_key

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [9]:
sourcefile='../WeatherPy/output_data/cityoutput.csv' 

# Read csv cities file and store into data frame:
city_data=pd.read_csv(sourcefile)
city_data.count()
city_data.head()




,City,Country,Date,Lat,Lng,Cloudiness (%),Humidity (%),Max Temp (F),Wind Speed (mph)
0,Rikitea,PF,1627950562,-23.1203,-134.9692,82,55,69.93,19.60
1,Busselton,AU,1627950580,-33.6500,115.3333,100,79,47.05,7.74
2,Linxia Chengguanzhen,CN,1627950645,35.6003,103.2064,100,88,59.05,4.99
3,Vaini,TO,1627950573,-21.2000,-175.2000,20,73,75.36,12.66
4,Zhicheng,CN,1627950645,30.2956,111.5047,7,79,85.19,8.23


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [3]:
max_hum=city_data['Humidity (%)'].max()
max_hum

100

In [31]:
gmaps.configure(api_key=g_key)

city_data=city_data.dropna() 
locations=city_data[['Lat', 'Lng']].astype(float)
humidityintensity=city_data['Humidity (%)'].astype(float)

fig=gmaps.figure(map_type='SATELLITE')
heat_layer=gmaps.heatmap_layer(locations, weights=humidityintensity, max_intensity=max_hum,point_radius=3)
fig.add_layer(heat_layer)
fig


Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [5]:
city_data_narrow=city_data[(city_data['Max Temp (F)'] < 80) &
                             (city_data['Max Temp (F)'] > 65) &
                             (city_data['Wind Speed (mph)'] < 10) &
                             (city_data['Cloudiness (%)'] < 15) &
                             (city_data['Humidity (%)'] < 30)]
city_data_narrow=city_data_narrow.dropna() 
city_data_narrow.count()
city_data_narrow.head()
# Set up lists to hold response info:
name=[]        
city=[]        
country=[]      
lat=[]          
lng=[]         

# Loop through the list of cities and perform a request for data on each and add to corresponding lists:

count=1 

print('Beginning Data Retrieval')     
print('-----------------------------')
for i in range(len(city_data_narrow)) : 
   
    lati=str(city_data_narrow.iloc[i, 3])    
    lngi=str(city_data_narrow.iloc[i, 4])
    
    target_coordinates=lati + ', ' + lngi  
    target_search='Hotel'
    target_radius=5000
    target_type='lodging'

   
    params={ 'location': target_coordinates,'keyword': target_search,'radius': target_radius,'type': target_type,'key': g_key}

    # Base url:
    base_url='https://maps.googleapis.com/maps/api/place/nearbysearch/json'
    
    response=requests.get(base_url, params=params)
   
    places_data=response.json()
    #print(json.dumps(places_data, indent=4, sort_keys=True))
    
    
    print('Looking for Hotels near: ' + str(city_data_narrow.iloc[i, 0]) + ' | Record ' + str(count) + ' of Total ' + str(len(city_data_narrow)))
     
    try:                                            
        name.append(places_data['results'][0]['name'])
        city.append(city_data_narrow.iloc[i, 0])
        country.append(city_data_narrow.iloc[i, 1])
        lat.append(places_data['results'][0]['geometry']['location']['lat'])
        lng.append(places_data['results'][0]['geometry']['location']['lng'])
    except IndexError:
        print('No Hotel Near Location... Skipping...') 
  
    count=count+1
    
# Print Ending Log Statement:
print('-------------------------------')
print('Data Retrieval Complete')
print('-------------------------------')

Beginning Data Retrieval
-----------------------------
Looking for Hotels near: Baghlan | Record 1 of Total 3
No Hotel Near Location... Skipping...
Looking for Hotels near: Chaman | Record 2 of Total 3
Looking for Hotels near: Moranbah | Record 3 of Total 3
-------------------------------
Data Retrieval Complete
-------------------------------


In [6]:
hotel_dict={'City': city,'Country': country,'Lat': lat,'Lng': lng,'Hotel Name': name}
hotel_df=pd.DataFrame(hotel_dict)
hotel_df.count()
hotel_df.head()

,City,Country,Lat,Lng,Hotel Name
0,Chaman,PK,30.928355,66.445181,Al khaleej hotel
1,Moranbah,AU,-22.002072,148.042501,Oaks Moranbah Suites


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [7]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]

In [8]:
# Add marker layer ontop of heat map
figure_layout={'width': '955px','height': '600px','border': '1px solid black','padding': '1px','margin': '0 auto 0 auto'}
fig=gmaps.figure(layout=figure_layout)

# Assign the marker layer to a variable:
markers=gmaps.marker_layer(locations, info_box_content=hotel_info)
# Add the layer to the map:
fig.add_layer(markers)
fig



Figure(layout=FigureLayout(border='1px solid black', height='600px', margin='0 auto 0 auto', padding='1px', wi…